#**Pre Proses**

In [ ]:
!git clone https://github.com/debajyoty/Mask-Face-recog-Project.git

Cloning into 'Mask-Face-recog-Project'...
remote: Enumerating objects: 2602, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2602 (delta 2), reused 0 (delta 0), pack-reused 2596
Receiving objects: 100% (2602/2602), 165.32 MiB | 31.18 MiB/s, done.
Resolving deltas: 100% (569/569), done.
Checking out files: 100% (2765/2765), done.


In [ ]:
cd ./Mask-Face-recog-Project/

/content/Mask-Face-recog-Project


In [ ]:
pip install face-recognition

     |████████████████████████████████| 100.1 MB 35 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=c71ccf1a3ad3d6e373e1567c9542ec8f9428f320da2f4b9b68b9868d886dbe98
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
!nvidia-smi

In [ ]:
cd Recog_Train/

In [ ]:
import os
os.remove('.DS_Store')

In [ ]:
cd ..

/content/Mask-Face-recog-Project


# **Import Library**

In [ ]:
#Base Library
import os
import cv2
import face_recognition
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import imutils
import time
import math
import sys
from threading import Timer
import shutil
import docopt
from sklearn import svm
#Video library
import PIL
import io
import html
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode

## State Image

In [ ]:
img_path = os.getcwd() + "/Recog_Train"

images = []
class_names = []
encode_list = []
encode_list_cl = []
myList = os.listdir(img_path)

#print(myList)

for subdir in myList:
  path = img_path + '/' + subdir + '/'
  for img in os.listdir(path):
    img_pic = path + img
    class_names.append(subdir)
    print("class names : ",img_pic)
    cur_img = cv2.imread(img_pic)
    cur_img = cv2.cvtColor(cur_img , cv2.COLOR_BGR2RGB)
    images.append(cur_img)

class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowipakemasker4.jpg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowi.jpeg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowi.jpg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowipakemasker.jpeg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowi.png
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowipakemasker.png
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowipakemasker.jpg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowi3.jpg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Jokowi/jokowi2.jpeg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Raffi/raffi.jpg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Raffi/raffi6.jpg
class names :  /content/Mask-Face-recog-Project/Recog_Train/Raffi/raffi4.jpeg
class names :  /conte

## Remove .ipnb_checkpoint
Sering terjadi error ketika mendelete image yaitu tidak terbacanya image, dikarenakan ada folder .ipnb_checkpoint yang mengganggu proses pengambilan/pengumpulan image ke state.
Running ketika terjadi error tersebut

In [ ]:
rmdir "/content/Mask-Face-recog-Project/Recog_Train/Shakira/.ipynb_checkpoints"

# Mask Detection function

In [ ]:
#plt.imshow()

#for img in images:
    

def detect_and_predict_mask(frame, faceNet, maskNet,threshold):
	# grab the dimensions of the frame and then construct a blob
	# from it
	global detections 
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))
	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []
	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence >threshold:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)
			face = np.expand_dims(face, axis=0)
            
			# add the face and bounding boxes to their respective
			# lists
			locs.append((startX, startY, endX, endY))
			# print("tes",maskNet.predict(face)[0].tolist())
			preds.append(maskNet.predict(face)[0].tolist())
	return (locs, preds)
# SETTINGS
MASK_MODEL_PATH="./masksdetection-master/model/mask_model.h5"
FACE_MODEL_PATH="./masksdetection-master/face_detector"
SOUND_PATH="./masksdetection-master/sounds/alarm.wav" 
THRESHOLD = 0.5

# Load Sounds
#mixer.init()
#sound = mixer.Sound(SOUND_PATH)
from os.path import dirname, join

protoPath = "./deploy.prototxt"
weightsPath = "./res10_300x300_ssd_iter_140000.caffemodel"
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
#prototxtPath = "C:\masksdetection-master\masksdetection-master\face_detector\deploy.prototxt.txt"
#weightsPath = os.path.sep.join([FACE_MODEL_PATH,"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(protoPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model(MASK_MODEL_PATH)

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
# vs = VideoStream(0).start()
# time.sleep(2.0)
        
#for cl in myList :
 #   cur_img = cv2.imread(f'{img_path} / {cl}' )
 #   images.append([cur_img])
        
def find_encodings(images) :
	for img in images:
		# heightFace,widthFace,_ = img.shape
		# face_location = (0,heightFace,widthFace,0)
		encodings = face_recognition.face_encodings(img)[0]
		# if(len(encodings)>0):
		# 	encode_list.append(encodings[0])
		# else:
		# 	print("No faces found in the image!")
		return encode_list
    
encodeListKnown = find_encodings(images)
 
# cap = cv2.VideoCapture(0)

# while True : 

		
		# key = cv2.waitKey(1) & 0xFF
		
		# if key == ord("q"):
		# 		break
# cv2.destroyAllWindows()
# vs.stop()

[INFO] loading face detector model...
[INFO] loading face mask detector model...
[INFO] starting video stream...


#**Helper Stream**

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

## Webcam Image Helper

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data) 
  
  # call our darknet helper on webcam image
  detections, width_ratio, height_ratio = darknet_helper(img, width, height)

  # loop through detections and draw them on webcam image
  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    cv2.rectangle(img, (left, top), (right, bottom), class_colors[label], 2)
    cv2.putText(img, "{} [{:.2f}]".format(label, float(confidence)),
                      (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                      class_colors[label], 2)
  # save image
  cv2.imwrite(filename, img)

  return filename

## Video Stream Helper

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# Detection Mask and face Recog

## Multiple Image Detection (Beta)
belum selesai jangan di running

In [ ]:
encodings = []
names = []
for person in myList:
  pix = os.listdir(img_path+"/"+person)
  for person_img in pix:
    # Get the face encodings for the face in each image file
    path = img_path+"/"+person+"/"+person_img
    face = face_recognition.load_image_file(path)
    face_bounding_boxes = face_recognition.face_locations(face)

    # If training image contains exactly one face
    if len(face_bounding_boxes) == 1:
        face_enc = face_recognition.face_encodings(face)[0]
        # Add face encoding for current image 
        # with corresponding label (name) to the training data
        encodings.append(face_enc)
        names.append(person)
    else:
        print(person + "/" + person_img + " can't be used for training")

clf = svm.SVC(gamma ='scale')
clf.fit(encodings, names)

# Load the test image with unknown faces into a numpy array
test_image = cv2.imread("./Detect/raffidanjokowi.jpg")
test_image = cv2.resize(test_image,(1000,1200),None,0.25,0.25)
test_image = cv2.cvtColor(test_image,cv2.COLOR_BGR2RGB)

# Find all the faces in the test image using the default HOG-based model
face_locations = face_recognition.face_locations(test_image)
no = len(face_locations)
print("Number of faces detected: ", no)

# Predict all the faces in the test image using the trained classifier

# facesCurFrame = face_recognition.face_locations(test_image,model="cnn")
# encodes  = face_recognition.face_encodings(test_image,face_locations)
(locs, preds) = detect_and_predict_mask(test_image, faceNet, maskNet,THRESHOLD)
print("Found:")
for pred,face_loc in zip(preds,face_locations):
    (mask,withoutMask) = pred
    for i in range(no):
      encodings = face_recognition.face_encodings(test_image)[i]
      name = clf.predict([encodings])
      label = "Mask" if mask > withoutMask else "No Mask"
      color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
      label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
      color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

      y1,x2,y2,x1 = face_loc
      # y1,x2,y2,x1 = y1*4 , x2*4 , y2*4 , x1*4
      cv2.rectangle(test_image,(x1,y1),(x2,y2),color,2)
      cv2.rectangle(test_image,(x1,y2-35),(x2,y2),color, cv2.FILLED)
      cv2.putText(test_image, *name, (x1+6 , y2 - 6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2 )
      cv2.putText(test_image, label, (x1 , y2+10) , cv2.FONT_HERSHEY_COMPLEX , 1, (255,0,255) , 2)
      cv2_imshow(test_image)
      print(*name)
      print(label)
    

## Old Image Detection (Alpha)
belum selesai janga di running

In [ ]:
# img = cv2.imread('./Detect/shakira.jpg')
img = cv2.imread("./Detect/shakira.jpg")
imgs = cv2.resize(img,(0,0),None,0.25,0.25)
imgs = cv2.cvtColor(imgs , cv2.COLOR_BGR2RGB)
(locs, preds) = detect_and_predict_mask(imgs, faceNet, maskNet,THRESHOLD)
facesCurFrame = face_recognition.face_locations(imgs,model="cnn")
encodes  = face_recognition.face_encodings(imgs,facesCurFrame)
encodeCurFrame = []
if(len(encodes)>0):
  encodeCurFrame.append(encodes[0])
else:
  print("no faces found in the image")

for encodeFace , faceLoc , pred in zip(encodeCurFrame,facesCurFrame ,preds):
  (mask, withoutMask) = pred
  label = "Mask" if mask > withoutMask else "No Mask"
  color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
  label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

  matches = face_recognition.api.compare_faces(encode_list, encodeFace)
  faceDis = face_recognition.api.face_distance(encode_list, encodeCurFrame)
  print("faceDis : ",faceDis)
  # matchIndex = np.argmin(faceDis)
  # print('index : ',matchIndex)
  name = class_names[0]
  print('nama : ',name)
  color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

  y1,x2,y2,x1 = faceLoc
  y1,x2,y2,x1 = y1*4 , x2*4 , y2*4 , x1*4
  cv2.rectangle(img,(x1,y1),(x2,y2),color,2)
  cv2.rectangle(img,(x1,y2-35),(x2,y2),color, cv2.FILLED)
  cv2.putText(img, name, (x1+6 , y2 - 6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2 )
  cv2.putText(img, label, (x1 , y2+10) , cv2.FONT_HERSHEY_COMPLEX , 1 , (255,0,255) , 2)
  
  plt.imshow(img)
  if(matches==True):
      print("yes is me")
      print()
  else:
      print("no me")
		
    

## Video Detection With Helper (Pre-Alpha)

## Encodings image
bisa di running

In [ ]:
encodings=[]
names = []
for person in myList:
  pix = os.listdir(img_path+"/"+person)
  for person_img in pix:
    # Get the face encodings for the face in each image file
    path = img_path+"/"+person+"/"+person_img
    face = face_recognition.load_image_file(path)
    face_bounding_boxes = face_recognition.face_locations(face)

    # If training image contains exactly one face
    if len(face_bounding_boxes) == 1:
        face_enc = face_recognition.face_encodings(face)[0]
        # Add face encoding for current image 
        # with corresponding label (name) to the training data
        encodings.append(face_enc)
        names.append(person)
    else:
        print(person + "/" + person_img + " can't be used for training")

clf = svm.SVC(gamma ='scale')
clf.fit(encodings, names)

Jokowi/jokowipakemasker.jpeg can't be used for training
Jokowi/jokowipakemasker.png can't be used for training
Raffi/raffi6.jpg can't be used for training


SVC()

## Stream
bisa di running

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # call our darknet helper on video frame
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet,THRESHOLD)
    # Find all the faces in the test image using the default HOG-based model
    face_locations = face_recognition.face_locations(frame)
    no = len(face_locations)
    print("Number of faces detected: ", no)
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet,THRESHOLD)
    print("Found:")

    # loop through detections and draw them on transparent overlay image
    for pred,face_loc in zip(preds,face_locations):
      (mask,withoutMask) = pred
      for i in range(no):
        encodings = face_recognition.face_encodings(frame)[i]
        name = clf.predict([encodings])
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

      y1,x2,y2,x1 = face_loc
      bbox_array = cv2.rectangle(bbox_array,(x1,y1),(x2,y2),color,2)
      bbox_array = cv2.rectangle(bbox_array,(x1,y2-35),(x2,y2),color, cv2.FILLED)
      bbox_array = cv2.putText(bbox_array, *name, (x1+6 , y2 - 6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2 )
      bbox_array = cv2.putText(bbox_array, label, (x1 , y2+10) , cv2.FONT_HERSHEY_COMPLEX , 1, (255,0,255) , 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

## Image Detection with helper (Pre-Alpha)
belom selesai jangan di running

In [ ]:
try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))